## Load Data

#### Using colab to load data:
#### These steps are ommited due to data privacy

## Import Libraries and Packages

In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [5]:
import keras
print(keras.__version__)

3.10.0


In [6]:
import pandas as pd
import numpy as np

## Loading the Dataset Files

In [4]:
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory= './train/',
    label_mode= 'categorical',
    labels= 'inferred',
    color_mode= 'rgb',
    shuffle= True,
    seed = 42,
    batch_size=64,
    image_size=(256, 256),
    validation_split=0.2,
    subset='training'
)

class_names = train_dataset.class_names
print('Class names:', class_names)



val_dataset = image_dataset_from_directory(
    directory= './train/',
    label_mode= 'categorical',
    labels= 'inferred',
    color_mode= 'rgb',
    shuffle= False,
    batch_size=64,
    image_size=(256, 256),
    validation_split=0.2,
    subset='validation'
)

Found 8829 files belonging to 11 classes.
Using 7064 files for training.
Class names: ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Found 8829 files belonging to 11 classes.
Using 1765 files for validation.


In [5]:
from keras.utils import image_dataset_from_directory

test_dataset = image_dataset_from_directory(
    directory= './test/',
    label_mode=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=False,
)

from pathlib import Path
test_files = [Path(path).stem for path in test_dataset.file_paths]
print(test_files)

Found 1201 files.
['-56lhw2AKjYI0Hnt', '-6OdHXCBItIArPyk', '-7241lsvPiVpNVFV', '-8-0wltLEZBDTM5M', '-GcrzANWUmrjk2tb', '-H3eW6fAmjabFQEv', '-HiiXK6MB6F1u40a', '-K5ftb4XBgVegcq7', '-PVsEsjibQBimlP0', '-Q0Yo_MTjnhzgtN9', '-TBD4tIe-a5UP1mh', '-Tktdo4X5LgUMvSw', '-U9lVuyU3rjePU2w', '-dDdmvuCoHRNvJK-', '-ga31aOTDSPAk8W0', '-jatHKBptixJPEQT', '-o4zntf0z1Eu0IJ6', '-oeDjNxexoilWeox', '-u1wASthhxChx3G5', '-vCVFJtJTs7YChq5', '-xAm9bKdi9qTCCra', '-xq0ILJth3DVCYt2', '-zJwe8YBxr85Kqg9', '02OB8CEAAq0oyBwZ', '02VCtYpDEvYkINWl', '03DAx7-4cWGuhEX5', '03xupnFJ2o_2kqKa', '0BgZnLHZLYz1FmtI', '0E9rEioGHMxjdCWj', '0EWPPBjWdTUI9GOE', '0Ec2a9JMPyN8E691', '0GhwqifywrmSbfZK', '0KbUyXdr0ZMjaFFm', '0L0u1A_aWPUVG5aH', '0RJ3OX2VBHElngIk', '0VAS0qijlcyjfSfW', '0Z5yioYpOoNH-iRv', '0jbb92ze77YgRYK7', '0r-YN2GLC7NQQcGp', '0ull0MeSHKGf3pJF', '0umwxpjgv3xtrzyJ', '0wyzNFWaH6guStX-', '0xg_-CaB8YpE3LBT', '0xthRPq5IQSSb0Xh', '0yyLOJ-IjDC9fv90', '1FRKnFTejuYLMB4w', '1G0SAiYEaHRA1ZRk', '1NlN_OKl40FQC3Pz', '1_b3eGKt9rlVkfQt', '

## Preprocessing

In [ ]:
from keras.applications.resnet50 import preprocess_input

train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x: preprocess_input(x))

In [ ]:
# save data in order the colab crash

X_train_dataset = np.concatenate([x.numpy() for x, y in train_dataset])
y_train_dataset = np.concatenate([y.numpy() for x, y in train_dataset])

X_val_dataset = np.concatenate([x.numpy() for x, y in val_dataset])
y_val_dataset = np.concatenate([y.numpy() for x, y in val_dataset])


np.save("X_train_dataset.npy", X_train_dataset)
np.save("y_train_dataset.npy", y_train_dataset)

np.save("X_val_dataset.npy", X_val_dataset)
np.save("y_val_dataset.npy", y_val_dataset)

## Design Deep Learning Model

In [ ]:
# use resnet50 for transfer learning

model = keras.applications.ResNet50(input_shape=(256, 256, 3),
                                    classes= 11,
                                    weights="imagenet",
                                    include_top=False,
                                    pooling="max")

model.trainable = False

x = keras.layers.Flatten()(model.output)

x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(11, activation='softmax')(x)
model = keras.models.Model(model.input, x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# set optimizer

OPTIMIZER = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=OPTIMIZER,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# set hyperparameters and callbacks

EPOCHS = 30
BATCH_SIZE = 256

CALLBACKS = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5,
                                           restore_best_weights=True),
             keras.callbacks.ModelCheckpoint("my_model1.keras", save_best_only=True)]


# fit model

history = model.fit(train_dataset,
                    epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_data=val_dataset,
                    callbacks=CALLBACKS)

Epoch 1/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 80s 527ms/step - accuracy: 0.6493 - loss: 1.1444 - val_accuracy: 0.9037 - val_loss: 0.2956
Epoch 2/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 44s 310ms/step - accuracy: 0.8386 - loss: 0.4861 - val_accuracy: 0.9099 - val_loss: 0.3002
Epoch 3/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 42s 373ms/step - accuracy: 0.8691 - loss: 0.3968 - val_accuracy: 0.9586 - val_loss: 0.1444
Epoch 4/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 38s 341ms/step - accuracy: 0.9053 - loss: 0.2873 - val_accuracy: 0.9258 - val_loss: 0.2259
Epoch 5/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 36s 299ms/step - accuracy: 0.9264 - loss: 0.2374 - val_accuracy: 0.9235 - val_loss: 0.2485
Epoch 6/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 39s 342ms/step - accuracy: 0.9215 - loss: 0.2225 - val_accuracy: 0.9184 - val_loss: 0.2686
Epoch 7/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 43s 362ms/step - accuracy: 0.9377 - loss: 0.1849 - val_accuracy: 0.9717 - val_loss: 0.0879
Epoch 8/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 37s 330ms/step - accuracy: 0.9480 - loss: 0

## Evaluate Model

In [ ]:
# if the colab crashed, run this block to load data

from tensorflow import keras
model = keras.models.load_model('my_model1.keras')

import numpy as np
X_train_dataset = np.load('X_train_dataset.npy')
y_train_dataset = np.load('y_train_dataset.npy')
X_val_dataset = np.load('X_val_dataset.npy')
y_val_dataset = np.load('y_val_dataset.npy')

In [2]:
from sklearn.metrics import accuracy_score

predictions = model.predict(X_val_dataset)
predictions = (predictions > 0.5).astype(int)
accuracy_score(y_val_dataset, predictions)

56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 205ms/step


0.9790368271954675

## Prediction

In [7]:
preds = model.predict(test_dataset)
preds_indices = np.argmax(preds, axis=1)
preds_classes = [int(class_names[pred]) for pred in preds_indices]

19/19 ━━━━━━━━━━━━━━━━━━━━ 23s 660ms/step


In [8]:
# Preparing submission file
import pandas as pd
submission = pd.DataFrame(columns=['image_name', 'cat_id'])
submission['image_name'] = test_files
submission['image_name'] = submission['image_name'].apply(lambda x: x + '.jpg')
submission['cat_id'] = preds_classes
submission

,image_name,cat_id
0,-56lhw2AKjYI0Hnt.jpg,0
1,-6OdHXCBItIArPyk.jpg,2
2,-7241lsvPiVpNVFV.jpg,9
3,-8-0wltLEZBDTM5M.jpg,9
4,-GcrzANWUmrjk2tb.jpg,8
...,...,...
1196,zNIgOhLGUPCyp7vl.jpg,2
1197,zhzjJeI8FwQDZQCE.jpg,1
1198,zizDow_ExDVnz9QS.jpg,2
1199,znpJGKcXxvmOU_nK.jpg,0


#### In the target column, each id refers to specific category of clothes:


| <b>category_id</b> | <b>category_title</b> |
| :---: | :---: |
| <code>0</code> | Men's Jackets, Raincoats, and Overcoats |
| <code>1</code> | Men's Sweatshirts and Hoodies |
| <code>2</code> | Analog and Digital Wristwatches |
| <code>3</code> | Wall, Desk, and Decorative Clocks |
| <code>4</code> | Accessories for Regular and Smart Watches |
| <code>5</code> | Kids' and Teens' Sweatshirts and Hoodies |
| <code>6</code> | Kids' and Teens' Jackets and Overcoats |
| <code>7</code> | Men's Sports Sweatshirts |
| <code>8</code> | Men's Sports Sweatshirts and Pants |
| <code>9</code> | Shopping Bags and Trolleys |
| <code>10</code> | Suitcases and Bags |